# Union Fitness record board
This page uses data from the OpenPowerlifting project, https://www.openpowerlifting.org. <br>
You may download a copy of the data at https://gitlab.com/openpowerlifting/opl-data.

##### This script generates the data source for the [Union Fitness record board](https://datastudio.google.com/u/0/reporting/feac28b1-2dd2-40f1-b141-d44f305e5235/page/ianP).

In [1]:
import pygsheets
import pandas as pd

from IPython.core.display import display, HTML
display(HTML('<style>.container { width:70% !important; }</style>'))

In [2]:
# import the members list from Google Sheets
gc = pygsheets.authorize(service_file='client_secret.json')
spreadsheet = gc.open_by_key('1F9p68_sX_dYViJZSBpbszMka6c2a0LpCsCXW8qdpNxY')
members = spreadsheet.worksheet_by_title('Members').get_as_df()

In [3]:
# import the data from OpenPowerlifting
data_types = {
              'Name': str,
              'Sex': str,
              'Equipment': str,
              'WeightClassKg': str,
              'Best3SquatKg': float,
              'Best3BenchKg': float,
              'Best3DeadliftKg': float,
              'Squat4Kg': float,
              'Bench4Kg': float,
              'Deadlift4Kg': float,
              'TotalKg': float,
              'Wilks': float,
              'Date': str,
              'MeetName': str,
              'Federation': str,
              'Place': str
             }
op = pd.read_csv('openpowerlifting-2020-02-15.csv', dtype=data_types, usecols=list(data_types))

In [4]:
# inner join the two dataframes
merged_df = pd.merge(op, members, on='Name')

In [5]:
# filter the data
mask = ((merged_df['Date'] >= merged_df['JoinDate'])
        & ((merged_df['Date'] <= merged_df['DepartureDate']) | merged_df['DepartureDate'].isna()) 
        & (merged_df['Place'] != 'DQ'))
uf_only = merged_df[mask].copy()

In [6]:
# account for 4th attempts
uf_only['BestSquat'] = uf_only[['Best3SquatKg', 'Squat4Kg']].max(axis=1)
uf_only['BestBench'] = uf_only[['Best3BenchKg', 'Bench4Kg']].max(axis=1)
uf_only['BestDeadlift'] = uf_only[['Best3DeadliftKg', 'Deadlift4Kg']].max(axis=1)

In [7]:
# convert kilograms to pounds
uf_only[['BestSquat', 'BestBench', 'BestDeadlift', 'Total']] = uf_only[['BestSquat', 'BestBench', 'BestDeadlift', 'TotalKg']].apply(lambda x: x*2.20462)

In [8]:
# reformat weight classes
def translate_weight_class(x):
    if x == '52':
        return '(01) -114 (M + F)'
    elif x == '56':
        return '(02) -123 (M + F)'
    elif x == '60':
        return '(03) -132 (M + F)'
    elif x == '63':
        return '(04) -138 (F)'
    elif x == '67.5':
        return '(05) -148 (M + F)'
    elif x == '72':
        return '(06) -158 (F)'
    elif x == '75':
        return '(07) -165 (M + F)'
    elif x == '82.5':
        return '(08) -181 (M + F)'
    elif x == '90':
        return '(09) -198 (M + F)'
    elif x == '90+':
        return '(10) 198+ (F)'
    elif x == '93':
        return '(11) -205 (M)'
    elif x == '100':
        return '(12) -220 (M)'
    elif x == '110':
        return '(13) -242 (M)'
    elif x == '120':
        return '(14) -264 (M)'
    elif x == '120+':
        return '(15) 264+ (M)'
    elif x == '125':
        return '(16) -275 (M)'
    elif x == '140':
        return '(17) -308 (M)'
    elif x == '140+':
        return '(18) 308+ (M)'
    else:
        return 'error'

uf_only['WeightClass'] = uf_only['WeightClassKg'].apply(translate_weight_class)

In [9]:
# remove hashtags from names
uf_only['Name'] = uf_only['Name'].str.replace(r' #\d', '')

In [10]:
# adjust some names
uf_only.loc[uf_only['Name'] == 'Francis Snyder', 'Name'] = 'Stew Snyder'
uf_only.loc[uf_only['Name'] == 'Edward Jones', 'Name'] = 'Eddy Jones'
uf_only.loc[uf_only['Name'] == 'Stephen Buccilli', 'Name'] = 'Steve Buccilli'

In [11]:
# prepare final data
columns_to_drop = [
                   'WeightClassKg', 
                   'Best3SquatKg', 
                   'Best3BenchKg', 
                   'Best3DeadliftKg', 
                   'Squat4Kg','Bench4Kg', 
                   'Deadlift4Kg', 
                   'TotalKg', 
                   'Place', 
                   'JoinDate', 
                   'DepartureDate'
                  ]
data = uf_only.drop(columns=columns_to_drop)
data['Date'] = pd.to_datetime(data['Date'])
data.drop_duplicates(inplace=True)
data.sort_values(by=['Date', 'MeetName', 'Name'], inplace=True)

In [12]:
# upload final data to Google Sheets
dashboard = spreadsheet.worksheet_by_title('Dashboard')
dashboard.clear()
dashboard.set_dataframe(data, (1,1), nan='')